Using Neural Networks and Latency mechanism[1] to spot cracks and reduce the number of false positives


[1]Enhanced Video-Level Anomaly Feature Detection for Nuclear Power Plant
Component Inspections Using the Latency Mechanism - Zhouxiang Fei, Graeme M. West, Paul Murray, Gordon Dobie

- using CNN (either GoogleLeNet or YOLOv8) to classify patches of an image
- seeing if the flagged patches remain for a certain number of frames
- flagging as crack if it does


In [1]:
!pip install torch opencv-python numpy torchvision pillow 


[notice] A new release of pip is available: 23.1.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl (204.2 MB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
  Using cached numpy-2.2.2-cp311-cp311-win_amd64.whl (12.9 MB)
  Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl (1.6 MB)
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl (2.6 MB)
  Using cached filelock-3.17.0-py3-none-any.whl (16 kB)
  Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
  Using cached jinja2-3.1.5-py3-none-any.whl (134 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl (184 kB)
  Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl (15 kB)


In [8]:
!.\crack_detection\Scripts\activate
!pip install ultralytics

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/914.6 kB ? eta -:--:--
   --------------------------------------- 914.6/914.6 kB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------------------------ --- 7.3/8.0 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 29.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------------------------- -------------- 7.3/11.6 MB 37.8 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.6 MB 31.4 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 26.9 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/43.9 MB ? 

In [15]:
import torch
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
from torch import nn
import torch
import torch.nn.functional as F
from ultralytics import YOLO
from collections import deque

test running model on single crack image

In [ ]:
model_path = "best.pt" 
model = YOLO(model_path).to("cpu")

def yolov8(image_path):
    image_path = "beam crack.jpg"  # Replace with your image path
    image = cv2.imread(image_path)

    results = model(image)

    # Display results
    for result in results:
        # Show the detected segmentation masks
        result.show()  # Opens a window with the detection results


0: 384x640 (no detections), 154.6ms
Speed: 0.0ms preprocess, 154.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)


: 

In [14]:
cap = cv2.VideoCapture(0) 

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    # Run YOLOv8 inference on the frame
    results = model.predict(frame)  # You can also pass a path to a frame image if you need
    
    # Process results (boxes, labels, and confidence scores)
    boxes = results[0].boxes  # Bounding box data for detected objects
    for box in boxes:
        # Draw bounding boxes around detected cracks
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get the coordinates (x1, y1, x2, y2) of the box
        conf = box.conf[0]  # Get the confidence of the detection
        
        # Draw a rectangle and label (optional)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(frame, f'{conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the resulting frame with detected cracks
    cv2.imshow("Crack Detection", frame)

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 297.3ms
Speed: 7.0ms preprocess, 297.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 204.6ms
Speed: 5.0ms preprocess, 204.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 200.1ms
Speed: 3.0ms preprocess, 200.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 208.4ms
Speed: 4.5ms preprocess, 208.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 196.0ms
Speed: 3.0ms preprocess, 196.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.6ms
Speed: 3.0ms preprocess, 201.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 192.9ms
Speed: 2.0ms preprocess, 192.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 205.3ms
Speed: 4.0ms prepr

In [16]:
# Load YOLO model
model_path = "best.pt"
model = YOLO(model_path).to("cpu")

# Video Capture
cap = cv2.VideoCapture("crack1.mp4")  # Use 0 for webcam, or replace with a video file path

# Parameters for the latency mechanism
frame_buffer = deque(maxlen=3)  # Store detections for the last 3 frames
latency_threshold = 3  # Number of consecutive detections required to confirm an anomaly
detection_history = {}  # Track detections across frames

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Run YOLO on the frame
    results = model.predict(frame)

    # Extract bounding boxes
    boxes = results[0].boxes
    current_frame_detections = []

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        conf = float(box.conf[0])  # Confidence score

        # Store current detections
        current_frame_detections.append((x1, y1, x2, y2))

    # Update the buffer
    frame_buffer.append(current_frame_detections)

    # Check for persistent detections
    for (x1, y1, x2, y2) in current_frame_detections:
        key = (x1, y1, x2, y2)  # Unique identifier for each detected box

        if key in detection_history:
            detection_history[key] += 1  # Increment count if detected again
        else:
            detection_history[key] = 1  # Initialize detection count

        # Draw the detection only if it appears in N consecutive frames
        if detection_history[key] >= latency_threshold:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            cv2.putText(frame, f'{conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    # Remove detections that are no longer seen
    for key in list(detection_history.keys()):
        if key not in current_frame_detections:
            detection_history[key] -= 1  # Reduce count if not detected in this frame
            if detection_history[key] <= 0:
                del detection_history[key]  # Remove if absent for too long

    # Display the frame
    cv2.imshow("Crack Detection", frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 231.0ms
Speed: 14.2ms preprocess, 231.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 156.1ms
Speed: 0.0ms preprocess, 156.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 130.8ms
Speed: 9.0ms preprocess, 130.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 128.3ms
Speed: 0.0ms preprocess, 128.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 138.4ms
Speed: 0.0ms preprocess, 138.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 136.9ms
Speed: 4.8ms preprocess, 136.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 135.9ms
Speed: 0.0ms preprocess, 135.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 131.2ms
Speed: 0.0ms prep